In [1]:
lot = df('magbc.sha.byhsa.LotRgmnReGroup')

In [2]:
lot.smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                 19052   23.17%       19052   23.17%
HER+AC                     346    0.42%       19398   23.59%
HER+FEC                    139    0.17%       19537   23.76%
HER+KAD                   1418    1.72%       20955   25.48%
HER+OTHER                 2720    3.31%       23675   28.79%
HER+PER+/-OTHER           4633    5.63%       28308   34.42%
HER+PER+KAD               1033    1.26%       29341   35.68%
HER+PER+TAXANE           19413   23.60%       48754   59.28%
HER+TAXANE                8002    9.73%       56756   69.01%
KAD                       2618    3.18%       59374   72.19%
KAD+OTHER                  307    0.37%       59681   72.57%
KAD+PER                     32    0.04%       59713   72.61%
OTHER                    11666   14.18%       71379   86.79%
PER+/-OTHER                 64    0.08%       71443   86.87%
PER+TAXANE                  40    0.05%

In [3]:
lot = lot.withColumn('lot_dur', datediff(col('lot_lst_dt'),col('lot_frst_dt')))

In [4]:
from pyspark.sql import Window

In [5]:
w = Window.partitionBy('ptnt_gid').orderBy('lot')

In [6]:
lot = lot.withColumn('nextLot', lead(col('lot_frst_dt')).over(w))

In [7]:
lots = lot.groupBy('ptnt_gid').agg(min('lot').alias('first_lot'), max('lot').alias('end_lot'), countDistinct('lot').alias('lotcnt'))

In [8]:
lot = lot.smvJoinByKey(lots, ['ptnt_gid'],'leftouter')

In [9]:
lot.smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                       18010   21.90%       18010   21.90%
2Lplus                   14818   18.02%       32828   39.92%
adjuvant                 41001   49.85%       73829   89.77%
neoadjuvant               8413   10.23%       82242  100.00%
-------------------------------------------------


In [10]:
firstlot = lot.smvGroupBy('ptnt_gid').smvTopNRecs(1, col('lot').asc())

In [11]:
firstlot = firstlot.filter((col('lot_frst_dt').smvYear() == 2013) | (col('lot_frst_dt').smvYear() == 2014))

In [12]:
firstlot.smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                        5541   20.52%        5541   20.52%
2Lplus                    3126   11.57%        8667   32.09%
adjuvant                 15484   57.33%       24151   89.42%
neoadjuvant               2858   10.58%       27009  100.00%
-------------------------------------------------


In [13]:
firstlot.count()

27009

In [14]:
early = firstlot.filter(col('lot')<1.0)

In [15]:
cohort1 = firstlot.filter(((col('new_lot_rgmn') == 'HER+PER+TAXANE') & (col('lot') == 0.1)) | ((col('new_lot_rgmn').like('HER%')) & (col('lot') == 0.2)))

In [16]:
cohort2 = firstlot.filter(((col('new_lot_rgmn') == 'HER+PER+/-OTHER') & (col('lot') == 1.0)) | ((col('new_lot_rgmn')=='KAD') & (col('lot') > 1.0)))

In [17]:
cohort1 = cohort1.select('ptnt_gid').distinct()

In [18]:
cohort1.count()

8950

In [19]:
cohort2 = cohort2.select('ptnt_gid').distinct()

In [ ]:
cohort2.count()

827

#### which stage do patients start in

In [ ]:
cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'leftouter').smvHist('cohort')

In [ ]:
cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'leftouter').smvHist('cohort')

#### lot dur

In [ ]:
cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').select('lot_dur').smvEdd()

In [ ]:
cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'inner').select('lot_dur').smvEdd()

#### diag2surg

In [ ]:
cohort1.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

In [ ]:
cohort2.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

In [ ]:
cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').withColumn('lotstr',col('end_lot').cast('string')).smvHist('lotstr')

In [ ]:
cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'inner').withColumn('lotstr',col('end_lot').cast('string')).smvHist('lotstr')

In [ ]:
cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

In [ ]:
cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'inner').withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

In [ ]:
group1 = cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').filter(col('cohort') == 'neoadjuvant').select('ptnt_gid').distinct()

In [ ]:
group2 = cohort1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').filter(col('cohort') == 'adjuvant').select('ptnt_gid').distinct()

In [ ]:
group3 = cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'inner').filter(col('cohort') == '1L').select('ptnt_gid').distinct()

In [ ]:
group4 = cohort2.smvJoinByKey(firstlot,['ptnt_gid'],'inner').filter(col('cohort') == '2Lplus').select('ptnt_gid').distinct()

In [ ]:
group1.smvJoinByKey(firstlot,['ptnt_gid'],'inner').select('lot_dur').smvEdd()

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == 'adjuvant').smvHist('new_lot_rgmn')

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == '1L').smvHist('new_lot_rgmn')

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot') == 2.0).smvHist('new_lot_rgmn')

In [ ]:
group3.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot') == 2.0).smvHist('new_lot_rgmn')

In [ ]:
lot.where(col('cohort') == 'adjuvant').smvHist('new_lot_rgmn')

# run above

In [8]:
first = lot.groupBy('ptnt_gid').agg(min(col('lot_frst_dt').smvYear()).alias('year'))

In [9]:
ptnts = first.where((col('year')==2013) | (col('year')==2014))

In [10]:
ptnts.count()

27009

In [11]:
ptnts = ptnts.smvJoinByKey(lot,['ptnt_gid'],'leftouter')

In [12]:
ptnts.withColumn('lotstr',col('first_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                       5742   14.46%        5742   14.46%
0.2                      19048   47.97%       24790   62.42%
1.0                       8966   22.58%       33756   85.00%
11.0                         1    0.00%       33757   85.00%
2.0                       3215    8.10%       36972   93.10%
3.0                       1425    3.59%       38397   96.69%
4.0                        730    1.84%       39127   98.53%
5.0                        280    0.71%       39407   99.23%
6.0                        170    0.43%       39577   99.66%
7.0                         86    0.22%       39663   99.88%
8.0                         19    0.05%       39682   99.92%
9.0                         30    0.08%       39712  100.00%
-------------------------------------------------


In [13]:
early = ptnts.where((col('first_lot')<1.0))

In [14]:
early.peek()

ptnt_gid:String                    = 00000000000073531261
year:Integer                       = 2014
lot:Decimal(10,1)                  = 0.1
ptnt_age:Integer                   = 54
ptnt_gender:String                 = F
ptnt_zip:String                    = 233
ptnt_st:String                     = VA
lot_frst_dt:Timestamp              = 2014-12-22 00:00:00.0
lot_lst_dt:Timestamp               = 2015-04-20 00:00:00.0
her_frst_dt:Timestamp              = 2014-12-22 00:00:00.0
her_lst_dt:Timestamp               = 2015-04-20 00:00:00.0
her_schedl:Integer                 = 7
tot_her_inf:Integer                = 9
tot_her_gap:Integer                = 119
tot_c_her_inf:Integer              = 8
tot_her_c_gap:Integer              = 63
her_drtn:Decimal(10,2)             = 18.00
her_cmplnt_drtn:Integer            = 64
per_frst_dt:Timestamp              = null
per_lst_dt:Timestamp               = null
tot_per_inf:Integer                = 0
tot_per_gap:Integer                = null
tot_gap_per_inf:I

In [31]:
ptnts.smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                        8477   21.35%        8477   21.35%
2Lplus                   10667   26.86%       19144   48.21%
adjuvant                 17710   44.60%       36854   92.80%
neoadjuvant               2858    7.20%       39712  100.00%
-------------------------------------------------


In [22]:
meta = ptnts.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where((col('cohort')=='1L') | (col('cohort')=='2Lplus'))

In [23]:
cohort1 = ptnts.where(((col('first_lot')=='neoadjuvant') & (col('new_lot_rgmn') == 'HER+PER+TAXANE')) | ((col('cohort')=='adjuvant') & (col('new_lot_rgmn').like('HER%'))))

In [24]:
cohort1.count()

11109

In [25]:
cohort2 = meta.where(((col('cohort')=='1L') & (col('new_lot_rgmn') == 'HER+PER+/-OTHER')) | ((col('cohort')=='2Lplus') & (col('new_lot_rgmn')=='KAD')))

In [26]:
cohort2.count()

1899

In [33]:
cohort1.withColumn('lotstr',col('first_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                       4128   37.16%        4128   37.16%
0.2                       6981   62.84%       11109  100.00%
-------------------------------------------------


In [41]:
c1neo = cohort1.where(col('cohort') == 'neoadjuvant')

In [42]:
c1adj = cohort1.where(col('cohort') == 'adjuvant')

In [40]:
c1g1 = cohort1.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == 'neoadjuvant')

In [39]:
c1g2 = cohort1.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == 'adjuvant')

In [36]:
c1g3 = cohort1.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == '1L')

In [37]:
c1g4 = cohort1.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort') == '2Lplus')

In [38]:
cohort1.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                        1782   10.23%        1782   10.23%
2Lplus                     779    4.47%        2561   14.71%
adjuvant                 10724   61.59%       13285   76.29%
neoadjuvant               4128   23.71%       17413  100.00%
-------------------------------------------------


In [43]:
cohort1.select('lot_dur').smvEdd()

lot_dur              Non-Null Count         11109
lot_dur              Null Count             0
lot_dur              Average                217.5544153389144
lot_dur              Standard Deviation     145.88940356685933
lot_dur              Min                    0.0
lot_dur              Max                    1532.0


## total population of patients

In [5]:
lot.peek()

ptnt_gid:String                    = 00000000000238490661
lot:Decimal(10,1)                  = 1.0
ptnt_age:Integer                   = 46
ptnt_gender:String                 = F
ptnt_zip:String                    = 078
ptnt_st:String                     = NJ
lot_frst_dt:Timestamp              = 2010-04-12 00:00:00.0
lot_lst_dt:Timestamp               = 2017-03-20 00:00:00.0
her_frst_dt:Timestamp              = 2010-04-12 00:00:00.0
her_lst_dt:Timestamp               = 2017-03-20 00:00:00.0
her_schedl:Integer                 = 21
tot_her_inf:Integer                = 118
tot_her_gap:Integer                = 2534
tot_c_her_inf:Integer              = 118
tot_her_c_gap:Integer              = 2534
her_drtn:Decimal(10,2)             = 365.00
her_cmplnt_drtn:Integer            = 2510
per_frst_dt:Timestamp              = null
per_lst_dt:Timestamp               = null
tot_per_inf:Integer                = 0
tot_per_gap:Integer                = null
tot_gap_per_inf:Integer            = null
tot_pe

In [10]:
lot.select('ptnt_gid').distinct().count()

59404

In [11]:
lot.where(col('lot') == 0.1).count()

8413

In [12]:
lott = df('magbc.sha.ptnt.LotDF')

In [13]:
lott.count()

169824

In [14]:
lott.select('lot_frst_dt').smvEdd()

lot_frst_dt          Time Start             "2007-01-02 00:00:00"
lot_frst_dt          Time Edd               "2017-03-31 00:00:00"
Histogram of lot_frst_dt: Year
key                      count      Pct    cumCount   cumPct
2007                      1360    0.80%        1360    0.80%
2008                     14958    8.81%       16318    9.61%
2009                     14242    8.39%       30560   18.00%
2010                     14158    8.34%       44718   26.33%
2011                     18062   10.64%       62780   36.97%
2012                     17966   10.58%       80746   47.55%
2013                     19340   11.39%      100086   58.94%
2014                     21605   12.72%      121691   71.66%
2015                     21608   12.72%      143299   84.38%
2016                     22726   13.38%      166025   97.76%
2017                      3799    2.24%      169824  100.00%
-------------------------------------------------
Histogram of lot_frst_dt: Month
key                    

# Identify different groups to track
## Split lot into four stages

In [15]:
neo = lot.where(col('cohort')=='neoadjuvant')

In [16]:
adj = lot.where(col('cohort')=='adjuvant')

In [17]:
one = lot.where(col('cohort')=='1L')

In [18]:
two = lot.where(col('cohort')=='2Lplus')

In [19]:
neo.count()

8413

In [20]:
adj.count()

41001

In [21]:
one.count()

18010

In [22]:
two.count()

14818

In [23]:
neo.smvJoinByKey(adj.select('ptnt_gid'),['ptnt_gid'], 'inner').count()

6237

In [24]:
adj.smvJoinByKey(one.select('ptnt_gid'),['ptnt_gid'], 'inner').count()

5015

In [25]:
lot.smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                       18010   21.90%       18010   21.90%
2Lplus                   14818   18.02%       32828   39.92%
adjuvant                 41001   49.85%       73829   89.77%
neoadjuvant               8413   10.23%       82242  100.00%
-------------------------------------------------


## Within each stage identify patients taking soc regimen

In [26]:
soc = neo.where(col('new_lot_rgmn') == 'HER+PER+TAXANE')

In [27]:
soc.select(col('lot_frst_dt').smvYear()).show()

+--------------------+
|SmvYear(lot_frst_dt)|
+--------------------+
|                2014|
|                2015|
|                2015|
|                2015|
|                2014|
|                2013|
|                2016|
|                2015|
|                2015|
|                2016|
|                2014|
|                2015|
|                2015|
|                2015|
|                2014|
|                2015|
|                2014|
|                2014|
|                2014|
|                2014|
+--------------------+
only showing top 20 rows



In [28]:
group2 = adj.filter(col('new_lot_rgmn').like('HER%'))

In [29]:
group2.count()

26359

In [29]:
td = df('magbc.sha.ptnt.TargetDates')

In [31]:
td.peek()

ptnt_gid:String                 = 00000000000850294928
First_BreastDiagDate:Timestamp  = 2015-02-23 00:00:00.0
First_Her2TestDate:Timestamp    = null
First_IHCTestDate:Timestamp     = null
First_ISHTestDate:Timestamp     = null
First_BRCATestDate:Timestamp    = null
First_TreamentDate:Timestamp    = 2015-04-02 00:00:00.0
First_Her2TargetDate:Timestamp  = 2015-04-02 00:00:00.0
First_ChemoDate:Timestamp       = 2015-04-02 00:00:00.0
First_HormoneDate:Timestamp     = null
First_SurgeryDate:Timestamp     = 2015-08-17 00:00:00.0
First_TrastuzumabDate:Timestamp = 2015-04-02 00:00:00.0
First_PertuzumabDate:Timestamp  = null
First_LapatinibDate:Timestamp   = null
First_TDM1Date:Timestamp        = null
Last_TreamentDate:Timestamp     = 2016-02-03 00:00:00.0
Last_Her2TargetDate:Timestamp   = 2016-02-03 00:00:00.0
Last_ChemoDate:Timestamp        = 2016-02-03 00:00:00.0
Last_HormoneDate:Timestamp      = null
Last_SurgeryDate:Timestamp      = 2015-08-17 00:00:00.0
Last_TrastuzumabDate:Timestamp  = 

In [34]:
ptnts.count()

54

In [39]:
group1 = soc.select('ptnt_gid')

In [40]:
group1.count()

6466

In [41]:
group3 = one.where(col('new_lot_rgmn') == 'HER+PER+/-OTHER')

In [42]:
group3.count()

1792

In [43]:
group4 = two.filter(col('new_lot_rgmn')=='KAD')

In [168]:
group4.count()

1621

In [169]:
group2.smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                 11931   45.26%       11931   45.26%
HER+AC                     162    0.61%       12093   45.88%
HER+FEC                     70    0.27%       12163   46.14%
HER+KAD                      1    0.00%       12164   46.15%
HER+OTHER                  351    1.33%       12515   47.48%
HER+PER+/-OTHER           1195    4.53%       13710   52.01%
HER+PER+TAXANE            6996   26.54%       20706   78.55%
HER+TAXANE                5653   21.45%       26359  100.00%
-------------------------------------------------


In [170]:
group2.peek()

ptnt_gid:String                    = 00000000000073531261
lot:Decimal(10,1)                  = 0.2
ptnt_age:Integer                   = 54
ptnt_gender:String                 = F
ptnt_zip:String                    = 233
ptnt_st:String                     = VA
lot_frst_dt:Timestamp              = 2015-05-18 00:00:00.0
lot_lst_dt:Timestamp               = 2015-12-14 00:00:00.0
her_frst_dt:Timestamp              = 2015-05-18 00:00:00.0
her_lst_dt:Timestamp               = 2015-12-14 00:00:00.0
her_schedl:Integer                 = 21
tot_her_inf:Integer                = 11
tot_her_gap:Integer                = 210
tot_c_her_inf:Integer              = 11
tot_her_c_gap:Integer              = 210
her_drtn:Decimal(10,2)             = 33.00
her_cmplnt_drtn:Integer            = 210
per_frst_dt:Timestamp              = null
per_lst_dt:Timestamp               = null
tot_per_inf:Integer                = 0
tot_per_gap:Integer                = null
tot_gap_per_inf:Integer            = null
tot_per_c_ga

In [171]:
tc = df('magbc.sha.ptnt.TargetClaims')

In [172]:
tc.peek()

clm_gid:String                     = 00002000840000481091
dt:Timestamp                       = 2016-06-14 00:00:00.0
svc_ln_seq:Integer                 = 1
ptnt_gid:String                    = 00000000000181032178
prodt:String                       = 71020
loc_cde:Integer                    = 4
diagArr:Array(String,true)         = WrappedArray(C50.919, Z01.818, , , , , , , , , , )
phyn_key:String                    = 00000000000001731160
phyn_gid:String                    = 00000000000001731160
clm_typ:String                     = PX_DX
clm_sts:String                     = null
day_sply:Integer                   = null
drg_id:String                      = null
adj_svc_unit_nbr:Integer           = 0
prodt_strgth:String                = null
qty:Decimal(38,7)                  = null
dt_mth:String                      = 201703
prodt_cd:String                    = Other
targetType:String                  = Procedure
procType:String                    = Radio
procSubType:String             

In [ ]:
one.peek()

In [174]:
one.smvJoinByKey(group1,['ptnt_gid'],'inner').select('ptnt_gid').show()

KeyboardInterrupt: 

In [ ]:
group3.select('ptnt_gid').show()

In [176]:
#look at members of group3 who were also in adjuvant

In [ ]:
x=group3.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort')=='adjuvant')

In [ ]:
x.smvHist('new_lot_rgmn')

# want to see progression of each group through stages

## count overlap between different groups

### group 1 patients reaching further stages

In [ ]:
group1.smvJoinByKey(adj,['ptnt_gid'],'inner').count()

In [ ]:
group1.smvJoinByKey(one,['ptnt_gid'],'inner').count()

In [ ]:
group1.smvJoinByKey(two,['ptnt_gid'],'inner').count()

### group 1 patients in later groups

In [ ]:
group1.smvJoinByKey(group2.select('ptnt_gid'),['ptnt_gid'],'inner').count()

In [ ]:
group1.smvJoinByKey(group3.select('ptnt_gid'),['ptnt_gid'],'inner').count()

In [ ]:
group1.smvJoinByKey(group4.select('ptnt_gid'),['ptnt_gid'],'inner').count()

### what regimen do group 1 patients take in later stages

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'inner').where(col('cohort')=='adjuvant').smvHist('new_lot_rgmn')

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'inner').where(col('cohort')=='1L').smvHist('new_lot_rgmn')

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'inner').where(col('cohort')=='2Lplus').smvHist('new_lot_rgmn')

### same for group 2

In [ ]:
group2.smvJoinByKey(one,['ptnt_gid'],'inner').count()

In [48]:
group2.smvJoinByKey(two,['ptnt_gid'],'inner').count()

1084

In [47]:
group2.smvJoinByKey(group4,['ptnt_gid'],'inner').count()

92

In [ ]:
group2.smvJoinByKey(group3.select('ptnt_gid'),['ptnt_gid'],'inner').count()

In [ ]:
group2.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'inner').where(col('cohort')=='1L').smvHist('new_lot_rgmn')

### group 3

In [ ]:
group3.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'],'inner').where(col('cohort')=='2Lplus').smvHist('new_lot_rgmn')

In [ ]:
group3.smvJoinByKey(two,['ptnt_gid'],'inner').count()

In [ ]:
group3.smvJoinByKey(group4.select('ptnt_gid'),['ptnt_gid'],'inner').count()

### group 4

In [ ]:
group4.select('ptnt_gid').smvJoinByKey(one,['ptnt_gid'],'inner').count()

## look at lot durations for each group

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'inner').select('lot_dur').smvEdd()

In [ ]:
neo.select('lot_dur').smvEdd()

In [ ]:
group2.select('lot_dur').smvEdd()

In [ ]:
adj.select('lot_dur').smvEdd()

In [ ]:
group3.select('lot_dur').smvEdd()

In [ ]:
one.select('lot_dur').smvEdd()

In [ ]:
group3.peek()

In [ ]:
group4.select('lot_dur').smvEdd()

In [ ]:
two.select('lot_dur').smvEdd()

## look at days from diagnosis til surgery for each group

In [ ]:
group1.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

In [ ]:
group2.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

In [ ]:
group3.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

In [ ]:
group4.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_SurgeryDate'), col('First_BreastDiagDate')).alias('diag2surg')).smvEdd()

## diagnosis til treatment

In [ ]:
group1.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_TreamentDate'), col('First_BreastDiagDate')).alias('diag2t')).smvEdd()

In [ ]:
group2.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_TreamentDate'), col('First_BreastDiagDate')).alias('diag2t')).smvEdd()

In [ ]:
group3.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_TreamentDate'), col('First_BreastDiagDate')).alias('diag2t')).smvEdd()

In [ ]:
group4.select('ptnt_gid').smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_TreamentDate'), col('First_BreastDiagDate')).alias('diag2t')).smvEdd()

## lot regimens for each stage

In [ ]:
adj.smvHist('new_lot_rgmn')

In [ ]:
neo.smvHist('new_lot_rgmn')

In [ ]:
one.smvHist('new_lot_rgmn')

In [ ]:
two.smvHist('new_lot_rgmn')

In [ ]:
group4.smvHist('neo_adj_rgmn')

In [ ]:
group3.smvHist('neo_adj_rgmn')

In [ ]:
group2.smvHist('rgrp_lot_rgmn')

In [ ]:
group3.smvHist('lot_frst_rgmn')

In [ ]:
lot.peek()

In [ ]:
group1.smvJoinByKey(td,['ptnt_gid'],'leftouter').where(col('First_Her2TestDate').isNotNull()).count()

In [ ]:
group2.smvJoinByKey(td,['ptnt_gid'],'leftouter').where(col('First_Her2TestDate').isNotNull()).count()

In [ ]:
group3.smvJoinByKey(td,['ptnt_gid'],'leftouter').where(col('First_Her2TestDate').isNotNull()).count()

In [ ]:
group4.smvJoinByKey(td,['ptnt_gid'],'leftouter').where(col('First_Her2TestDate').isNotNull()).count()

In [ ]:
group1.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_Her2TestDate'),col('First_BreastDiagDate')).alias('diff')).smvEdd()

In [ ]:
group2.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_Her2TestDate'),col('First_BreastDiagDate')).alias('diff')).smvEdd()

In [ ]:
group3.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_Her2TestDate'),col('First_BreastDiagDate')).alias('diff')).smvEdd()

In [ ]:
group4.smvJoinByKey(td,['ptnt_gid'],'leftouter').select(datediff(col('First_Her2TestDate'),col('First_BreastDiagDate')).alias('diff')).smvEdd()

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').select(datediff(col('nextLot'),col('lot_lst_dt')).alias('lot_diff')).smvEdd()

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('cohort')=='neoadjuvant').select(datediff(col('nextLot'),col('lot_lst_dt')).alias('lot_diff')).smvEdd()

In [ ]:
group2.select(datediff(col('nextLot'),col('lot_lst_dt')).alias('lot_diff')).smvEdd()

In [ ]:
group3.select(datediff(col('nextLot'),col('lot_lst_dt')).alias('lot_diff')).smvEdd()

In [ ]:
group4.select(datediff(col('nextLot'),col('lot_lst_dt')).alias('lot_diff')).smvEdd()

In [ ]:
lot.where(col('ptnt_gid')=='00000000000072582563').select('lot','lot_frst_dt','lot_lst_dt','cohort').show()

In [ ]:
lot.smvHist('cohort')

In [ ]:
group1.smvJoinByKey(group2,['ptnt_gid'],'inner').smvJoinByKey(group3,['ptnt_gid'],'inner').count()

In [ ]:
group4.where(col('lot')>4.0).count()

In [ ]:
group1.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot')>4.0).count()

In [ ]:
group3.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot')>4.0).count()

In [ ]:
group2.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot')>4.0).count()

In [ ]:
group4.smvJoinByKey(lot,['ptnt_gid'],'leftouter').where(col('lot')>4.0).count()

### where do they start and end

In [27]:
group2.withColumn('lotstr',col('end_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.2                      23023   87.34%       23023   87.34%
1.0                       2535    9.62%       25558   96.96%
2.0                        586    2.22%       26144   99.18%
3.0                        161    0.61%       26305   99.80%
4.0                         43    0.16%       26348   99.96%
5.0                          8    0.03%       26356   99.99%
6.0                          3    0.01%       26359  100.00%
-------------------------------------------------


In [36]:
group3.withColumn('lotstr',col('end_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
1.0                       1307   72.94%        1307   72.94%
2.0                        349   19.48%        1656   92.41%
3.0                        101    5.64%        1757   98.05%
4.0                         29    1.62%        1786   99.67%
5.0                          5    0.28%        1791   99.94%
6.0                          1    0.06%        1792  100.00%
-------------------------------------------------


In [28]:
adj.withColumn('lotstr',col('end_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.2                      35986   87.77%       35986   87.77%
1.0                       3782    9.22%       39768   96.99%
2.0                        878    2.14%       40646   99.13%
3.0                        267    0.65%       40913   99.79%
4.0                         68    0.17%       40981   99.95%
5.0                         14    0.03%       40995   99.99%
6.0                          5    0.01%       41000  100.00%
7.0                          1    0.00%       41001  100.00%
-------------------------------------------------


In [29]:
group2.where(col('first_lot') == 0.2).count()

20263

In [33]:
group2.withColumn('lotstr',col('first_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                       6096   23.13%        6096   23.13%
0.2                      20263   76.87%       26359  100.00%
-------------------------------------------------


In [34]:
group3.withColumn('lotstr',col('first_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                        166    9.26%         166    9.26%
0.2                        610   34.04%         776   43.30%
1.0                       1016   56.70%        1792  100.00%
-------------------------------------------------


In [35]:
group4.withColumn('lotstr',col('first_lot').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                         13    0.80%          13    0.80%
0.2                        125    7.71%         138    8.51%
1.0                        650   40.10%         788   48.61%
2.0                        453   27.95%        1241   76.56%
3.0                        224   13.82%        1465   90.38%
4.0                         91    5.61%        1556   95.99%
5.0                         42    2.59%        1598   98.58%
6.0                         13    0.80%        1611   99.38%
7.0                          8    0.49%        1619   99.88%
8.0                          1    0.06%        1620   99.94%
9.0                          1    0.06%        1621  100.00%
-------------------------------------------------


In [23]:
lot.where(col('first_lot') == 0.2).count()

41148

In [24]:
lot = lot.withColumn('lotstr',col('lot').cast('string'))

In [25]:
lot.smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
0.1                       8413   10.23%        8413   10.23%
0.2                      41001   49.85%       49414   60.08%
1.0                      18010   21.90%       67424   81.98%
10.0                        27    0.03%       67451   82.02%
11.0                        15    0.02%       67466   82.03%
12.0                         5    0.01%       67471   82.04%
13.0                         2    0.00%       67473   82.04%
14.0                         1    0.00%       67474   82.04%
15.0                         1    0.00%       67475   82.04%
16.0                         1    0.00%       67476   82.05%
17.0                         1    0.00%       67477   82.05%
18.0                         1    0.00%       67478   82.05%
19.0                         1    0.00%       67479   82.05%
2.0                       7514    9.14%       74993   91.19%
20.0                         1    0.00%      

In [49]:
soc.withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
1                         1067   16.50%        1067   16.50%
2                         5165   79.88%        6232   96.38%
3                          202    3.12%        6434   99.51%
4                           28    0.43%        6462   99.94%
5                            4    0.06%        6466  100.00%
-------------------------------------------------


In [41]:
group2.withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
1                        17181   65.18%       17181   65.18%
2                         8160   30.96%       25341   96.14%
3                          774    2.94%       26115   99.07%
4                          185    0.70%       26300   99.78%
5                           45    0.17%       26345   99.95%
6                           11    0.04%       26356   99.99%
7                            3    0.01%       26359  100.00%
-------------------------------------------------


In [42]:
group3.withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
1                          683   38.11%         683   38.11%
2                          822   45.87%        1505   83.98%
3                          217   12.11%        1722   96.09%
4                           58    3.24%        1780   99.33%
5                           11    0.61%        1791   99.94%
7                            1    0.06%        1792  100.00%
-------------------------------------------------


In [43]:
group4.withColumn('lotstr',col('lotcnt').cast('string')).smvHist('lotstr')

Histogram of lotstr: String sort by Key
key                      count      Pct    cumCount   cumPct
1                          284   17.52%         284   17.52%
2                          646   39.85%         930   57.37%
3                          431   26.59%        1361   83.96%
4                          183   11.29%        1544   95.25%
5                           60    3.70%        1604   98.95%
6                           16    0.99%        1620   99.94%
7                            1    0.06%        1621  100.00%
-------------------------------------------------


In [44]:
group1.smvJoinByKey(lot,['ptnt_gid'], 'inner').where(col('cohort')=='1L').smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                   227   35.80%         227   35.80%
HER+AC                      41    6.47%         268   42.27%
HER+FEC                     28    4.42%         296   46.69%
HER+OTHER                   17    2.68%         313   49.37%
HER+PER+/-OTHER             99   15.62%         412   64.98%
HER+PER+KAD                  1    0.16%         413   65.14%
HER+PER+TAXANE             119   18.77%         532   83.91%
HER+TAXANE                  11    1.74%         543   85.65%
KAD                         14    2.21%         557   87.85%
OTHER                       77   12.15%         634  100.00%
-------------------------------------------------


In [45]:
group1.smvJoinByKey(lot,['ptnt_gid'], 'inner').where(col('cohort')=='2Lplus').smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                    30   17.75%          30   17.75%
HER+AC                      11    6.51%          41   24.26%
HER+FEC                      8    4.73%          49   28.99%
HER+KAD                      6    3.55%          55   32.54%
HER+OTHER                   14    8.28%          69   40.83%
HER+PER+/-OTHER             24   14.20%          93   55.03%
HER+PER+KAD                  7    4.14%         100   59.17%
HER+PER+TAXANE              13    7.69%         113   66.86%
HER+TAXANE                   2    1.18%         115   68.05%
KAD                          6    3.55%         121   71.60%
KAD+OTHER                    3    1.78%         124   73.37%
OTHER                       42   24.85%         166   98.22%
TCH                          3    1.78%         169  100.00%
-------------------------------------------------


In [54]:
group2.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'], 'inner').where(col('cohort')=='1L').smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                   956   28.66%         956   28.66%
HER+AC                      20    0.60%         976   29.26%
HER+FEC                     13    0.39%         989   29.65%
HER+KAD                     11    0.33%        1000   29.98%
HER+OTHER                  175    5.25%        1175   35.22%
HER+PER+/-OTHER            563   16.88%        1738   52.10%
HER+PER+KAD                  3    0.09%        1741   52.19%
HER+PER+TAXANE             889   26.65%        2630   78.84%
HER+TAXANE                 175    5.25%        2805   84.08%
KAD                        157    4.71%        2962   88.79%
KAD+OTHER                    5    0.15%        2967   88.94%
KAD+PER                      1    0.03%        2968   88.97%
OTHER                      336   10.07%        3304   99.04%
PER+/-OTHER                  5    0.15%        3309   99.19%
TCH                         27    0.81%

In [52]:
group2.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'], 'leftouter').smvHist('cohort')

Histogram of cohort: String sort by Key
key                      count      Pct    cumCount   cumPct
1L                        3336    9.05%        3336    9.05%
2Lplus                    1084    2.94%        4420   11.99%
adjuvant                 26359   71.48%       30779   83.47%
neoadjuvant               6096   16.53%       36875  100.00%
-------------------------------------------------


In [55]:
group2.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'], 'inner').where(col('cohort')=='2Lplus').smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                   138   12.73%         138   12.73%
HER+AC                       2    0.18%         140   12.92%
HER+FEC                      1    0.09%         141   13.01%
HER+KAD                     87    8.03%         228   21.03%
HER+OTHER                  101    9.32%         329   30.35%
HER+PER+/-OTHER            176   16.24%         505   46.59%
HER+PER+KAD                121   11.16%         626   57.75%
HER+PER+TAXANE             117   10.79%         743   68.54%
HER+TAXANE                  58    5.35%         801   73.89%
KAD                         92    8.49%         893   82.38%
KAD+OTHER                   20    1.85%         913   84.23%
KAD+PER                      1    0.09%         914   84.32%
OTHER                      161   14.85%        1075   99.17%
PER+/-OTHER                  1    0.09%        1076   99.26%
TCH                          8    0.74%

In [56]:
group3.select('ptnt_gid').smvJoinByKey(lot,['ptnt_gid'], 'inner').where(col('cohort')=='2Lplus').smvHist('new_lot_rgmn')

Histogram of new_lot_rgmn: String sort by Key
key                      count      Pct    cumCount   cumPct
HER mono                    39    5.88%          39    5.88%
HER+KAD                     47    7.09%          86   12.97%
HER+OTHER                   38    5.73%         124   18.70%
HER+PER+/-OTHER            207   31.22%         331   49.92%
HER+PER+KAD                114   17.19%         445   67.12%
HER+PER+TAXANE              59    8.90%         504   76.02%
HER+TAXANE                  21    3.17%         525   79.19%
KAD                         81   12.22%         606   91.40%
KAD+OTHER                   13    1.96%         619   93.36%
OTHER                       39    5.88%         658   99.25%
PER+/-OTHER                  3    0.45%         661   99.70%
TCH                          2    0.30%         663  100.00%
-------------------------------------------------
